This file contains the code to find the most suitable K value for the KNN filling algorithm
Since Logistic regression performs and worst among all algorithm, we don't use it here

In [1]:

from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da
import dask_ml
import joblib

import dask
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.csvcsv)
import sklearn as sk
# import statsmodels.api as sm
# # Cross Validation Classification Accuracy
# from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold,cross_val_score
# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing 
# from sklearn.impute import KNNImputer
# #import matplotlib.pyplot as plot
# from matplotlib import pyplot
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score,f1_score
from sklearn import tree
# import statistics
import time
import warnings
warnings.filterwarnings('ignore')
# plotting errorbar graph for visual representation of the performance of the classifiers
# import matplotlib.pyplot as plt
# from matplotlib.transforms import Affine2D
import random
from xgboost import XGBClassifier
# import shutil
# import os
from our_evaluate_sepsis_score import evaluate_performance_dask
import our_functions_library as flib

In [2]:
# only run this once
client = Client()

In [3]:
client

Client Scheduler: tcp://127.0.0.1:51929 Dashboard: http://127.0.0.1:51927/status,Cluster Workers: 4 Cores: 16 Memory: 16.95 GB


In [4]:
# Only run this when the client has started
client.shutdown()

In [5]:
client.close()

In [4]:
# dataType={'Age': 'int', 'Gender': bool, 'Patient_id': int, 'time': 'int8', 'SepsisLabel':'float16'}
filename = '../../raw_data/raw_data_400.csv' # use raw dataset
originalData = dd.read_csv(filename,dtype={'Gender':bool,'SepsisLabel':bool}) # read csv data into Dask DataFrame
Uniq_ID= np.unique(originalData['Patient_id'])
print("Data size:",originalData.shape)
print('Patient id size:',len(Uniq_ID))
print(Uniq_ID)

X_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_columns = ['Patient_id', 'SepsisLabel']

# To test if these 3 columns is useful
# originalData = originalData.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)
# X_columns.remove('Unit1')
# X_columns.remove( 'Unit2')
# X_columns.remove( 'EtCO2')

# Initialize the empty array X_train, X_test, y_train, y_test
# X_train = dd.from_pandas(pd.DataFrame(columns = X_columns),npartitions=2)
# X_test = dd.from_pandas(pd.DataFrame(columns = X_columns),npartitions=2)
# y_train = dd.from_pandas(pd.DataFrame(columns = y_columns),npartitions=2)
# y_test = dd.from_pandas(pd.DataFrame(columns = y_columns),npartitions=2)

# Below are the lists for KNN results of different K value
KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []
KNN_F1Score_mean = []
KNN_F1Score_std = []
KNN_auroc_mean = []
KNN_auprc_mean = []
KNN_accuracy_mean = []
KNN_accuracy_std = []
KNN_positiveprediction_mean = []
KNN_baseline_mean = [ ]
KNN_total_Time= [ ]
fillmethod =""  

Data size: (Delayed('int-e1f3c4f3-c685-4295-9183-2280926a3427'), 43)
Patient id size: 400
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 2

In [5]:
originalData.compute().shape

(15348, 43)

In [6]:
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 2
random.seed(seed)
if(np.array_equal(range(1,21),Uniq_ID[:20]) ):
    print("The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs 
#     Uniq_ID = da.from_array(Uniq_ID)
else:
    print("They are not the same. Uniq_id has already been shuffled.")    
print("The original uniq id set is:\n",Uniq_ID[0:20])
print("The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49...\n",Uniq_ID[:20])

The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.
The original uniq id set is:
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204]
The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49...
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204]


In [7]:
def KNN_reset():
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()
    

def generateTrainDataSet(test_patienIds):
    train_data= originalData[~(originalData.Patient_id.isin(test_patienIds))]
    X_train = train_data[X_columns]
    y_train = train_data[y_columns] 
    return X_train, y_train


def generateTestDataSet(patienIds):
    print("test_patienIds: \n", patienIds)
    test_data = originalData[originalData['Patient_id'].isin(patienIds)]
    X_test = test_data[X_columns]
    y_test = test_data[y_columns]
    return X_test, y_test


def KNNfilling(trainData,testData,K= 5):
    imputer = KNNImputer(n_neighbors = K)
    #imputer = FaissKNeighbors(k=K)
    imputer.fit(trainData)
    x_train_impute=imputer.transform(trainData).round(3).astype('float32')
    x_test_impute=imputer.transform(testData).round(3).astype('float32')
#     x_train_impute=imputer.transform(trainData)
#     x_test_impute=imputer.transform(testData)
    fillmethod= "KnnFill"
    return x_train_impute, x_test_impute,fillmethod

In [11]:
model=sk.tree.DecisionTreeClassifier(random_state=2)
KforKFold=10
KforKNN=5
fillmethod=""
# Code for K-fold algorithm 
# def KFold_patient(model, KforKFold=10,KforKNN=5, fillmethod=""):
global X_train, X_test, y_train, y_test
print("Datasize",len(Uniq_ID) ,"with K Fold of ",KforKFold ," folds filling by KNN =",KforKNN)

# initialisation of the array for storing the different intermediate results
auroc_model = []
auprc_model = []
physio_accuracy_model = []
f_measure_model = []
utility_score_model = []
positivepredictions = []
baseline_model= []
#The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups
#This for loop is for Kfold, calculating the results for K times

start = time.time()# time indicator for how long the Kfold func takes

#write the idset data into a file
open("test.txt", "w").close() # clear contents of existing file
for i in range(KforKFold):
    splitted = " ".join( repr(e) for e in idSets[i])
    file1 = open("test.txt","a")
    file1.write("\n\n")
    file1.write(str("[" +splitted+"]"))
    file1.write("\n\n")
    file1.close()
print("Time for writing id to file:", round(time.time()- start,3))

# for i in range(KforKFold):
for i in range(1):
    print("for the",i+1,"th iteration: ")
    tfold = time.time()
    X_test,y_test = generateTestDataSet(idSets[i])
    print("Time for splitting id to test dataset:", round(time.time()- tfold,3))
    t1 = time.time()
    X_train, y_train = generateTrainDataSet(idSets[i])
    print("Time for splitting id to train dataset:", round(time.time()- t1,3))

    #Now the train and test dataset is generated 
    patientID_ytest = y_test['Patient_id']
    y_test = y_test.drop('Patient_id', 1) 
    y_train = y_train.drop('Patient_id', 1)
#     X_train=X_train.astype('float16')
#     X_test=X_test.astype('float16') 
    X_train=X_train.astype('float32')
    X_test=X_test.astype('float32') 
    y_train=y_train.astype('bool')
    y_test=y_test.astype('bool')
    print(X_train)
    
    #fill the missing data
    if X_train.isnull().values.any() or X_test.isnull().values.any() :
        t1= time.time()
        print("X_train or X_test contains NaN values, KNN is performed.")
#         X_train, X_test,fillmethod = flib.MeanFilling(X_train,X_test, fillmethod)  
        X_train, X_test, fillmethod = flib.KNNfilling(X_train, X_test, KforKNN)
        print("The type of X_train after KNN filling is",type(X_train))
        print("Time for NaN values filling: ", round(time.time()- t1,2) )
        X_train = dd.from_array(X_train)
        X_test = dd.from_array(X_test)
#         if np.isnan(X_train).any() or np.isnan(X_test).any() :
#             print("X_train_impute or X_test_impute still contains NaN values")          
        print("The type of X_train after KNN filling is",type(X_train))
        print(X_train.memory_usage(deep=True).sum().compute())
        print(X_test.memory_usage(deep=True).sum().compute())

    if ( type(model).__name__ == 'LogisticRegression'):  #Scale the data for logistic regression
        t1 = time.time()
        scaler = dask_ml.preprocessing.StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        print("Standard Scaling time: ", round(time.time()- t1,2) )

               
    #fit the model and predict
    with joblib.parallel_backend("dask",scatter=[X_train, y_train]):
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test).astype(bool)   
        y_predicted_probobility = model.predict_proba(X_test).astype('float32') 
    y_labels = y_test['SepsisLabel'].reset_index(drop=True) 
    del X_train, X_test, y_train

    print("columns of y_predicted_probobility is: ",len(y_predicted_probobility[0]))
    if(len(y_predicted_probobility[0]) ==2 ):
        y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
    elif(len(y_predicted_probobility[0]) == 1):     
        y_predicted_probobility =  y_predicted_probobility.round(4)
    
    auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance_dask(y_labels.compute(), y_predicted, y_predicted_probobility,patientID_ytest.compute(),idSets[i]) 
    del y_labels, y_predicted_probobility
    auroc_model.append(round(auroc,4))
    auprc_model.append(round(auprc,4))
    physio_accuracy_model.append(round(physio_accuracy,4))
    f_measure_model.append(round(f_measure,4))
    utility_score_model.append(round(utility_score,4))
    positivepredictions.append(np.sum(y_predicted))        
    baseline_model.append(round( (1 - y_test.mean().compute() )*100 , 2 ))
    print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
    del  y_test
    print("Time spent in this KFold iteration",round(time.time()-tfold,2),"sec.\n")
    print("******************************************************************")    
print("\nEvaluation parameters of the utiltiy evaluation function:")
print('auroc of model:', auroc_model)
print('auprc of model:', auprc_model)
print('utility F1 of model:', f_measure_model)
print('Utility accuracy of model:', physio_accuracy_model) 
print('Utility score of model:', utility_score_model)
print("Positive Prediction: ", positivepredictions)
print('Baseline model:', baseline_model)  
KNN_auroc_mean.append(np.mean(auroc_model))
KNN_auprc_mean.append(np.mean(auprc_model))
KNN_F1Score_mean.append(np.mean(f_measure_model))
KNN_F1Score_std.append(np.std(f_measure_model))
KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
KNN_accuracy_std.append(np.std(physio_accuracy_model))
KNN_UtilityScore_mean.append(np.mean(utility_score_model))
KNN_UtilityScore_std.append(np.std(utility_score_model))
KNN_positiveprediction_mean.append(np.mean(positivepredictions))
KNN_baseline_mean.append(np.mean(baseline_model))
totalTime=round(time.time()- start,2)
KNN_total_Time.append(totalTime)
print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")

Datasize 400 with K Fold of  10  folds filling by KNN = 5
Time for writing id to file: 0.002
for the 1 th iteration: 
test_patienIds: 
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204 157 290 305 281 323  57 208  41 366 220 120 118 244  22 325 222
 178 369 381  52]
Time for splitting id to test dataset: 0.004
Time for splitting id to train dataset: 0.003
Dask DataFrame Structure:
                    HR    O2Sat     Temp      SBP      MAP      DBP     Resp    EtCO2 BaseExcess     HCO3     FiO2       pH    PaCO2     SaO2      AST      BUN Alkalinephos  Calcium Chloride Creatinine Bilirubin_direct  Glucose  Lactate Magnesium Phosphate Potassium Bilirubin_total TroponinI      Hct      Hgb      PTT      WBC Fibrinogen Platelets      Age   Gender    Unit1    Unit2 HospAdmTime   ICULOS Patient_id     time
npartitions=1                                                                                                                                               

In [ ]:
# print(type(y_labels.compute()))
# print(y_labels)
# y_labels.compute().rename("labels")
y_test.compute().mean()

In [ ]:
from sys import getsizeof
print(getsizeof(X_train))
print(getsizeof(X_test))
print(X_train.shape)
print(X_test.shape)
# X_test.isnull().values.any()

In [ ]:
# print("X_train size is: ",X_train.size * X_train.itemsize)
# print("X_test size is: ",X_test.size * X_test.itemsize)
print("X_train size is: ",X_train.memory_usage(deep=True).sum().compute())
print("X_test size is: ",X_test.memory_usage(deep=True).sum().compute())
print("y_train size is: ",y_train.memory_usage(deep=True).sum().compute())
print("y_test size is: ",y_test.memory_usage(deep=True).sum().compute())

In [ ]:
np.sum(y_labels)
print(y_labels)
print(y_predicted)
print(type(y_predicted))
print(y_predicted_probobility)
print(type(y_predicted_probobility))

In [27]:
# X_train.loc[0:0].compute()
# X_train[0:0].compute()
# del y_train,y_test,X_test
# del y_test

In [ ]:
def Print_DATA():
    print("Numbers of K:",len(KNN_accuracy_mean))
    print("KNN_UtilityScore_mean", KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std", KNN_UtilityScore_std)
    print("KNN_F1Score_mean", KNN_F1Score_mean)
    print("KNN_F1Score_std", KNN_F1Score_std)
    print("KNN_auroc_mean", KNN_auroc_mean)
    print("KNN_auprc_mean", KNN_auprc_mean)
    print("KNN_accuracy_mean", KNN_accuracy_mean)
    print("KNN_accuracy_std", KNN_accuracy_std)
    print("KNN_positiveprediction_mean", KNN_positiveprediction_mean)
    print("KNN_baseline_mean", KNN_baseline_mean)
    print("KNN_total_time", KNN_total_Time)      

# def findBestKforKNN(model, KforKFold=10,KforKNNstart=1,KforKNNend=10, stepsize=5):
#     if(KforKNNend<=1):
#         print("K must be a interger larger than 1")
#         return
#     KNN_reset()
#     step=stepsize
#     for i in range(KforKNNstart,KforKNNend+1,step) : 
#         print("KNN of K = ",i)    
#         KFold_patient(model,10,i)

In [ ]:
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2)
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2, n_jobs=4)
#AdaBoostModel = AdaBoostClassifier(random_state=2)

In [ ]:
#fill the missing data with one function
print(filename)
#print(originalData)
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func')
#data,fillmethod = flib.linearFillingAll(originalData,1, True)#forwardfilling + fill other missing data with 0

In [ ]:
# KFold_patient(decisionTreeModel,10,5)
Print_DATA()